In [2]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
from dash.dependencies import Input, Output

# CSV data into DataFrames
df_words = pd.read_csv('word_analysis.csv')
df_reviews = pd.read_csv('reviews_sentiments.csv')

# get company names for dropdown options
company_options = [{'label': company, 'value': company} for company in df_words['Company Name'].unique()]
company_options.append({'label': 'All', 'value': 'All'})  # Add "All" option

# get the list of word buckets
bucket_columns = df_words.columns[4:]  # Columns containing word bucket counts

# Initialize the Dash app
app = dash.Dash(__name__)

# layout of the app
app.layout = html.Div([
    html.Div([
        dcc.Dropdown(id='company-dropdown', options=company_options, value=['All'], multi=True),
        dcc.Graph(id='review-sentiment-bar-chart'),
        dcc.Graph(id='sentiment-bar-chart'),
    ])
])

# color mappings for sentiment labels
color_discrete_map = {
    'Negative': 'red',
    'Neutral': 'blue',
    'Positive': 'green'
}

# Callback to update the sentiment bar chart
@app.callback(
    Output('sentiment-bar-chart', 'figure'),
    [Input('company-dropdown', 'value')]
)
def update_sentiment_bar_chart(selected_companies):
    if 'All' in selected_companies:
        filtered_df = df_words
    else:
        filtered_df = df_words[df_words['Company Name'].isin(selected_companies)]
    
    sentiment_scores = filtered_df.groupby('Sentiment Label')[bucket_columns].sum().T
    
    sorted_sentiment_scores = (
        sentiment_scores.sort_values(by=['Negative', 'Neutral', 'Positive'], ascending=False)
    )
    
    fig = px.bar(
        sorted_sentiment_scores,
        x=sorted_sentiment_scores.index,
        y=sorted_sentiment_scores.columns,
        title='Sentiment Scores by Word Buckets',
        labels={'x': 'Word Buckets', 'y': 'Word Count'},
        category_orders={"x": bucket_columns},  # Maintain order of word buckets on x-axis
        color_discrete_map=color_discrete_map,  # Specify color mapping for sentiment labels
        barmode='stack',  # Stack bars
    )
    
    fig.update_layout(xaxis_title="Word Buckets", yaxis_title="Word Count")  # Update axis titles
    
    return fig

# Callback to update the review sentiment bar chart
@app.callback(
    Output('review-sentiment-bar-chart', 'figure'),
    [Input('company-dropdown', 'value')]
)
def update_review_sentiment_bar_chart(selected_companies):
    if 'All' in selected_companies:
        filtered_df = df_reviews
    else:
        filtered_df = df_reviews[df_reviews['Company_name'].isin(selected_companies)]
    
    sentiment_percentage = (
        filtered_df.groupby(['Company_name', 'Sentiment_label'])['Review_ID']
        .count() / len(filtered_df) * 100
    ).reset_index()
    
    sorted_sentiment_percentage = (
        sentiment_percentage.sort_values(by='Review_ID', ascending=False)
    )
    
    fig = px.bar(
        sorted_sentiment_percentage,
        x='Review_ID',  # Count of distinct review IDs
        y='Company_name',
        color='Sentiment_label',
        title='Reviews per Company per Sentiment Label',
        labels={'x': 'Sum of Reviews', 'y': 'Company Name'},
        color_discrete_map=color_discrete_map,
        barmode='stack',  # Stack bars
        orientation='h'  # Horizontal bars
    )
    
    fig.update_layout(xaxis_title="Sum of Reviews")  # Update x-axis title
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)